In [ ]:
import os

from utils import calc_3d_pmapper, scale_descriptors

from sklearn.model_selection import train_test_split

from miqsar.estimators.wrappers import MIWrapperMLPRegressor, miWrapperMLPRegressor
from miqsar.estimators.attention_nets import AttentionNetRegressor
from miqsar.estimators.mi_nets import MINetRegressor, miNetRegressor

from sklearn.metrics import r2_score

Read data

In [ ]:
dataset_file = 'data/data.smi'

os.mkdir('descriptors')

path = os.path.join('descriptors', 'tmp')
os.mkdir(path)

Set hyperparameters

In [ ]:
n_epoch = 1000
batch_size = 128
lr = 0.001
weight_decay = 0.001
seed = 42
init_cuda = True
ncpu = 5
random_state = 42

Calculate descriptors for multiple conformations

In [ ]:
bags, labels, molid = calc_3d_pmapper(dataset_file, nconfs=50, stereo=False, path=path, ncpu=ncpu)
x_train, x_test, y_train, y_test, molid_train, molid_test = train_test_split(bags, labels, molid, random_state=random_state)
x_train, x_test = scale_descriptors(x_train, x_test)

Train MI model

In [ ]:
# train 3D/MI/Bag-AttentionNet
ndim = (x_train[0].shape[-1], 256, 128, 64)
det_ndim = (64,)

att_net = AttentionNetRegressor(ndim=ndim, det_ndim=det_ndim, init_cuda=init_cuda)
att_net.fit(x_train, y_train, n_epoch=n_epoch, batch_size=batch_size, weight_decay=weight_decay, lr=lr)

predictions = att_net.predict(x_test)
conformation_weights = att_net.get_instance_weights(x_test)

print('3D/MI/Bag-AttentionNet: r2_score = {:.2f}'.format(r2_score(y_test, predictions)))